In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy.stats import zscore


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
train = pd.read_csv('../dataset/train.csv')
test  = pd.read_csv('../dataset/test.csv')
org   = pd.read_csv("../dataset/jm1.csv")   # Original dataset from where train & test were extracted. The competition allows us to use it.



In [33]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

X = train.drop("defects", axis=1)
y = train["defects"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=972, stratify=y)

In [36]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_iter': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    # Add other hyperparameters to tune here
}
hist_gb_classifier = HistGradientBoostingClassifier(random_state=42)



In [37]:
grid_search = GridSearchCV(estimator=hist_gb_classifier, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=HistGradientBoostingClassifier(random_state=42),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'max_iter': [100, 200, 300]})

In [38]:
best_params = grid_search.best_params_

In [39]:
best_params

{'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 100}

In [41]:
best_hist_gb_classifier = grid_search.best_estimator_


In [42]:
y_pred = best_hist_gb_classifier.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


Accuracy: 0.8119288591922964
Classification Report:
               precision    recall  f1-score   support

       False       0.84      0.94      0.88      7870
        True       0.64      0.39      0.48      2307

    accuracy                           0.81     10177
   macro avg       0.74      0.66      0.68     10177
weighted avg       0.79      0.81      0.79     10177



In [43]:
# Prepare submission for Kaggle
# Predict test set to send to Kaggle
y_testKaggle =  best_hist_gb_classifier.predict(test)
y_testKaggle

array([False, False,  True, ..., False, False,  True])

In [45]:
df  = pd.read_csv('../dataset/test.csv', usecols=['id'])    # Prepare the id column from the original test csv
df['defects'] = pd.Series(y_testKaggle) # Add the defects column with the results

df.to_csv("../results/Binary software defect_thebesthistgradientboost.csv", index=False)